# Age Estimation and Gender Classification

In this assignment, you will train CNN models to estimate a person's age and predict their gender based on face images. Please read carefully all the instructions before you start to write your code.

**Your Tasks**

You are supposed to train two CNN models.
- Model A: Train a CNN model from scratch, following a few restrictions. Save this model as `age_gender_A.keras`
- Model B: Fine-tune a pre-trained model to perform the same task. Save this model as `age_gender_B.keras`

**Dataset**

Your models will be trained and validated on a folder `train_val/` containing 5,000 labeled face images (size: 128 x 128), originated from the UTKFace dataset. During the assessment phase, your code will be tested on unseen test data.

**Performance Metric**

The evaluation of your models on the test set will be based on the following metrics:
- age estimation: - Mean Absolute Error (MAE)
- gender prediction: accuracy

**GPU Time Management**

Please use the available GPU time wisely, as there are some limitations on free GPU usage. It is said the users can use up to 12 hours in row in Google Colab. But many people found they reached its limit far less than 12 hours. To minimize the risk of reaching the limit, consider the following suggestions:
1. Ensure you have a stable internet connection.
2. Carefully validate your hyperparameters before starting the training process.
3. Based on past experience, each model should be trained in less than 2 hours. If the training time is significantly longer, you may need to adjust your model architecture.


## Setting Up: Mount the google drive
Mount your google drive to the notebook. Also don't forget to **enable GPU** before your training.


In [ ]:
#
# Add your code here
#

# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

## Visualize a few photos
It's always beneficial to thoroughly understand your data before beginning the training process. To get a first-hand impression of the dataset, please display a minimum of 20 images from the `train_val/` folder, along with their corresponding age and gender labels. This will help you gain initial insights into the dataset.

In [ ]:
#
# Add your code here
#

# Relevant imports
import random
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Extracting the age and gender from the images
age = []
gender = []
race = []
datetime = []

# Folder containing the images
folder = '/content/drive/MyDrive/train_val'

# Loop through the files and extract the information from the file names
for images in os.listdir(folder):
  labels = images.split('_')
  age.append(labels[0])
  gender.append(labels[1])
  race.append(labels[2])
  datetime.append(labels[3].split('.')[0])

# Selecting a random subset of images to display
rand = [random.randint(0,5000) for i in range(20)]

# Viewing 20 random images
plt.figure(figsize=(10,10))
for i in range(20):
  plt.subplot(4,5,i+1)
  files = folder + '/' + str(age[rand[i]]) + '_' + str(gender[rand[i]]) + '_' + str(race[rand[i]]) + '_' + str(datetime[rand[i]]) + '.jpg.chip.jpg'
  img = imread(files)
  plt.imshow(img)
  plt.axis
  plt.xlabel(f'Age: {age[rand[i]]}, Gender: {gender[rand[i]]}')

## Step 1: Data Pre-Processing
In this step, you will perform necessary pre-processing tasks before feeding the data into a CNN. The pre-processing includes:

1. Splitting the data into training and validation sets.
2. Loading image data, gender labels and age values, and rescaling pixel values (originally integers between 0 and 255) to the range [0, 1].
3. Data augmentation

The first two pre-processing steps have already been completed for you. Your task in this step is limited to:

3. Data augmentation

Please note:

- Do not modify any part of the provided code except the section marked with ########.
- Do not rescale the age values to the range [0,1], as the original values are used during the testing stage. Rescaling them would significantly impact testing performance.



In [ ]:
import os
import random
import numpy as np

# Added new imports
import cv2
from tensorflow.keras import layers, Sequential

folder_train_val = '/content/drive/MyDrive/train_val'
all_image_files = [file for file in os.listdir(folder_train_val) if file.lower().endswith(('.jpg'))]

# Shuffle the dataset to ensure random distribution
random.seed(0)  # Ensure reproducibility
random.shuffle(all_image_files)

# Calculate the number of images for each set
n_train_val = len(all_image_files)
train_end = int(n_train_val * 0.8)

# Split the dataset
train_image_files = all_image_files[:train_end]
val_image_files = all_image_files[train_end:]
print(f"Number of images for training: {len(train_image_files)}")
print(f"Number of images for validation: {len(val_image_files)}")

def load_imgs_lables(dataset_path,filenames):
  print('load all image data, age and gender labels...')
  images = []
  age_labels = []
  gender_labels = []
  for current_file_name in filenames:
    img = cv2.imread(os.path.join(dataset_path, current_file_name))
    img = img / 255.0  # Normalize pixel values
    labels = current_file_name.split('_')
    age_label = int(labels[0])
    gender_label = int(labels[1])
    age_labels.append(age_label)
    gender_labels.append(gender_label)
    images.append(img)

  # Convert to NumPy arrays
  images = np.array(images)
  age_labels = np.array(age_labels)
  gender_labels = np.array(gender_labels)
  return images, age_labels, gender_labels

# Load data from the training set
train_images, train_age, train_gender = load_imgs_lables(folder_train_val,train_image_files)

# Load data from the validation set
val_images, val_age, val_gender = load_imgs_lables(folder_train_val,val_image_files)

#
######### Add your code for data augmentation here #########
#

# Data Augmentation Layer
data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
])

In [ ]:
# Visualize some of the augmented images

# Select a random subset of images to display
num_images_to_show = 5
sample_indices = random.sample(range(len(train_images)), num_images_to_show)
sample_images = train_images[sample_indices]

plt.figure(figsize=(12, 6))

for i, img in enumerate(sample_images):
    augmented_img = data_augmentation(np.expand_dims(img, 0))
    augmented_img_np = augmented_img[0].numpy()

    # Convert from BGR to RGB for displaying with matplotlib (Issue with Matplotlib turning images blue)
    rgb_img = cv2.cvtColor((augmented_img_np * 255).astype(np.uint8), cv2.COLOR_BGR2RGB)

    plt.subplot(1, num_images_to_show, i + 1)
    plt.imshow(rgb_img)
    plt.axis('off')

plt.tight_layout()
plt.show()

## STEP2A: Build your own CNN network
In this step, you will define your own CNN model for identifying gender and predicting age. Although there are two tasks, you only need to build one CNN model with two outputs. You may refer to online resources for solutions.

Here are the key requirements and restrictions for your network:
1.	The input size must be 128 x 128 x 3, which means you **should not resize** the original images. This is because my test code relies on this particular input size. Any other size will cause problem in the testing stage.
2.  Please treat the gender classification as a binary problem, i.e., **the output layer for the gender branch has only 1 unit**, instead of 2. Although it is technically correct to treat gender classification as a multi-class problem with 2 classes, i.e., 2 units in the last layer, the test code only supports a 1-unit last-layer configuration for the gender branch.
3.	The size of feature maps being fed to the first fully connected layer must be less than 10 x 10, while there is no number limitation about the depth.
4.	Incorporate techniques to prevent overfitting as appropriate.

At the end of your implementation, use `modelA.summary()` to display the model architecture and use `plot_model()` to visualize the architecture.

In [ ]:
#
# Add your code here
#

# Necessary imports
import keras.utils
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from keras.models import Model
from keras.regularizers import l2

# Initialize learning rate and input shape
l2_val = 0.01
input_shape = (128, 128, 3)

# Define model architecture
base = Sequential([

    # Convolutional layer 1
    layers.Conv2D(16, (3, 3), use_bias=False, kernel_regularizer=l2(l2_val), input_shape=input_shape, strides=1, padding = 'same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    # Convolutional layer 2
    layers.Conv2D(32, (3, 3), use_bias=False, kernel_regularizer=l2(l2_val), input_shape=input_shape, strides=1, padding = 'same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),

    # Convolutional layer 3
    layers.Conv2D(64, (2, 2), use_bias=False, kernel_regularizer=l2(l2_val), strides = 1, padding='valid'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
])

# Define the input and apply data augmentation
input = Input(shape=input_shape)
input = data_augmentation(input)
shared_layers = base(input)

# Gender classification branch

# Flatten the output from the shared layers
gender_flatten = layers.Flatten()(shared_layers)

# Dense layer for Gender
gender = Dense(64, use_bias=False, kernel_regularizer=l2(0.01))(gender_flatten)
gender1 = BatchNormalization()(gender)
gender2 = Activation('relu')(gender1)

# Adding Dropout
gender3 = Dropout(0.5)(gender2)

# Output layer for Gender
gender_output = Dense(1, activation='sigmoid', name='gender')(gender3)

# Age regression branch

# Addtional convolutional layer 1 for Age
age = layers.Conv2D(128, (3, 3), use_bias=False, kernel_regularizer=l2(l2_val), strides = 1)(shared_layers)
age = layers.BatchNormalization()(age)
age = layers.Activation('relu')(age)
age = layers.MaxPooling2D((2, 2))(age)

# Addtional convolutional layer 2 for Age
age = layers.Conv2D(256, (3, 3), use_bias=False, kernel_regularizer=l2(l2_val), strides=1)(age)
age = layers.BatchNormalization()(age)
age = layers.Activation('relu')(age)
age = layers.MaxPooling2D((2, 2))(age)

# Flatten the output from the extra convolutional layers
age = layers.Flatten()(age)

# Dense layer 1 for Age
age = Dense(512, use_bias=False, kernel_regularizer=l2(0.01))(age)
age1 = BatchNormalization()(age)
age2 = Activation('relu')(age1)

# Dense layer 2 for Age
age3 = Dense(128, use_bias=False, kernel_regularizer=l2(0.01))(age2)
age4 = BatchNormalization()(age3)
age5 = Activation('relu')(age4)

# Adding Dropout
age6 = Dropout(0.4)(age5)

# Output layer for Age
age_output = Dense(1, activation='relu', name='age')(age6)

# Create the model
modelA = Model(inputs=inputs, outputs=[gender_output, age_output])

# compile the model
modelA.summary()
from tensorflow.keras.utils import plot_model
plot_model(modelA, show_shapes=True,dpi=100)

## STEP3A: Compile and train your model
Compile and train your model here.
Save your model by `modelA.save(your_model_folder+"age_gender_A.keras")` after training.

**DON'T use any other name for your model file.** This is because my test code relies on this particular model name. Any other file name will cause problem in the testing stage.

**Save the model with `save()`** to save the entire model (including a model's architecture, weights, and training configuration) as a `.keras` zip archive. Don't save it with `save_weights()`.


In [ ]:
#
# Add your code here
#

# Necessary imports
from keras.callbacks import EarlyStopping, LearningRateScheduler

# Define the learning rate scheduler
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * np.exp(-0.1*epoch)

learning_rate_scheduler = LearningRateScheduler(scheduler)

# Define the early stopping criteria
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Compile the model
modelA.compile(loss={'gender': 'binary_crossentropy', 'age': 'mae'}, metrics={'gender': 'accuracy','age': 'mae'})

# Train the model
history1 = modelA.fit(
    x=train_images,
    y=[train_gender, train_age],
    validation_data=(val_images, [val_gender, val_age]),
    batch_size=32,
    epochs=30,
    callbacks=[learning_rate_scheduler, early_stop],
    )

In [ ]:
# Evaluate the model
modelA.evaluate(val_images, [val_gender, val_age])

In [ ]:
# Save the model
folder_models = '/content/drive/MyDrive/Models/'

modelA.save(folder_models+'age_gender_A.keras')

## STEP4A: Draw the learning curves
Draw four learning curve figures for model A as follows.
1.	The loss of the gender classification over the training and validation set
2.	The accuracy of the gender classification over the training and validation set
3.	The loss of the age estimation over the training and validation set
4.	The MAE of the age estimation over the training and validation set

In [ ]:
#
# Add your code here
#

# Get the history of the model
history_dict = history1.history

# Gender classification loss
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(history_dict['gender_loss'], label='Train Gender Loss')
plt.plot(history_dict['val_gender_loss'], label='Val Gender Loss')
plt.title('Gender Classification Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Gender classification accuracy
plt.subplot(2, 2, 2)
plt.plot(history_dict['gender_accuracy'], label='Train Gender Accuracy')
plt.plot(history_dict['val_gender_accuracy'], label='Val Gender Accuracy')
plt.title('Gender Classification Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Age estimation loss
plt.subplot(2, 2, 3)
plt.plot(history_dict['age_loss'], label='Train Age Loss')
plt.plot(history_dict['val_age_loss'], label='Val Age Loss')
plt.title('Age Estimation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Age estimation MAE
plt.subplot(2, 2, 4)
plt.plot(history_dict['age_mae'], label='Train Age MAE')
plt.plot(history_dict['val_age_mae'], label='Val Age MAE')
plt.title('Age Estimation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.tight_layout()
plt.show()

## STEP2B: Build a CNN network based on a pre-trained model
Choose an existing CNN architecture pre-trained on ImageNet and fine-tune it on this dataset.

As with Model A, **do not resize the input image size**.
Ensure that the output layer for **the gender branch has only 1 unit**.

At the end of the cell, use `modelB.summary()` to display the model architecture and use `plot_model()` to visualize the architecture.


## STEP3B: Compile and train your model
Compile and train your model here.
Save your model to `age_gender_B.keras` after training.

**DON'T use any other name for your model file.** This is because my test code relies on this particular model name. Any other file name will cause problem in the testing stage.

**Also, save the model with `save()` instead of `save_weights()`.**


In [ ]:
#
# Add your code here
#

# Necessary imports
from keras.applications import VGG16

# Load base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Update L2 value
l2_val = 0.001

# Freeze certain layers
base_model.trainable = False

for layer in base_model.layers[-3:]:
    layer.trainable = True

# Create input layer
input = Input(shape=(128, 128, 3))

# Apply augmentation layer
input = data_augmentation(input)

# Feed input into VGG16
shared_layer = base_model(x, training = False)

# Flatten the outputs
gender = Flatten()(shared_layer)
age = Flatten()(shared_layer)

# Dense layers for Gender
gender = Dense(256, use_bias=False, kernel_regularizer=l2(0.01), activation = 'relu')(gender)
gender = Dense(512, use_bias=False, kernel_regularizer=l2(0.01), activation = 'relu')(gender)

# Output layer for Gender
gender_output = Dense(1, activation='sigmoid', name='gender')(gender)

# Dense layers for Age
age = Dense(256, use_bias=False, kernel_regularizer=l2(0.01), activation = 'relu')(age)
age = Dense(512, use_bias=False, kernel_regularizer=l2(0.01), activation = 'relu')(age)

# Output layer for Gender
age_output = Dense(1, activation='relu', name='age')(age)

# Define model
modelB = Model(inputs=inputs, outputs=[gender_output, age_output])

# Compile the model
modelB.summary()
from tensorflow.keras.utils import plot_model
plot_model(modelB, show_shapes=True,dpi=100)

## STEP4B: Draw the learning curves
Draw four learning curve figures for model B as follows.
1. The loss of the gender classification over the training and validation set
2. The accuracy of the gender classification over the training and validation set
3. The loss of the age estimation over the training and validation set
4. The MAE of the age estimation over the training and validation set

In [ ]:
#
# Add your code here
#

# Define the optimizer
opt = keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model
modelB.compile(optimizer = opt,loss={'gender': 'binary_crossentropy', 'age': 'mae'}, metrics={'gender': 'accuracy','age': 'mae'})

# Train the model
history2 = modelB.fit(
    x=train_images,
    y=[train_gender, train_age],
    validation_data=(val_images, [val_gender, val_age]),
    batch_size=32,
    epochs=30,
    callbacks=[early_stop]
    )

In [ ]:
# Evaluate the model
modelB.evaluate(val_images, [val_gender, val_age])

In [ ]:
# Save the model
modelB.save(folder_models+'age_gender_B.keras')

In [ ]:
#
# Add your code here
#

# Get the history of the model
history_dict = history2.history

# Gender classification loss
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(history_dict['gender_loss'], label='Train Gender Loss')
plt.plot(history_dict['val_gender_loss'], label='Val Gender Loss')
plt.title('Gender Classification Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Gender classification accuracy
plt.subplot(2, 2, 2)
plt.plot(history_dict['gender_accuracy'], label='Train Gender Accuracy')
plt.plot(history_dict['val_gender_accuracy'], label='Val Gender Accuracy')
plt.title('Gender Classification Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Age estimation loss
plt.subplot(2, 2, 3)
plt.plot(history_dict['age_loss'], label='Train Age Loss')
plt.plot(history_dict['val_age_loss'], label='Val Age Loss')
plt.title('Age Estimation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Age estimation MAE
plt.subplot(2, 2, 4)
plt.plot(history_dict['age_mae'], label='Train Age MAE')
plt.plot(history_dict['val_age_mae'], label='Val Age MAE')
plt.title('Age Estimation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.tight_layout()
plt.show()

## STEP5: Evaluate the model on the test set
I will load your two models for testing. The test set will not be available before your submission.

The metrics for measuring the performance on the test set are:
- age estimation: MAE (Mean Absolute Error)
- gender classification: accuracy


In [ ]:
#
# Don't add code in this cell when submitting this file
#